In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from binerizer import *
from DNFRuleModel import DNFRuleModel
from MasterModel import MasterModel
from RuleGenerator import RuleGenerator
from Classifier import Classifier

In [2]:
credit = pd.read_csv('data/credit.csv')
data = binerizeData(credit, verbose = False)
X = data.to_numpy()[:,0:164]
Y = data.to_numpy()[:,165]

#X[Y,0] = 1
#X[~Y,0] = 0

mu = np.ones(len(X[Y,1]))
lam = 0

In [3]:
features = [1,2]
np.dot((np.all(X[:,features],axis=1) == Y)[Y], mu)

201.0

In [4]:
def computeObj(X, Y, features, lam, mu):
    classPos = np.all(X[:,features],axis=1)
    return lam*len(features) - np.dot(classPos[Y],mu) + sum(classPos[~Y])

In [11]:
tmp = np.zeros(X.shape[1])
tmp[[1,3,58]] = 1
tmp

array([0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
res = []
for i in range(X.shape[1]):
    res.append(computeObj(X, Y, [i], lam, mu))
ordered_indices = np.argsort(res)

feature_sets = []
good_rules = []
num_enums = 0
while num_enums < 200:
    for j in ordered_indices:
        for i in ordered_indices:
            if j in [i]:
                continue
            feature_sets.append([i,j])
            obj = computeObj(X, Y, [i,j], lam, mu)
            res.append(obj)
            if obj < 0:
                
                good_rules.append([i,j])
            num_enums += 1
    

KeyboardInterrupt: 

In [12]:
feature_set = [[]]
good_rules = []
K = 5
numKeep = 20
good_rule_obj = []

for i in range(K):
    newFeatures = []
    res = []
    for f in feature_set:
        print(f)
        print(len(feature_set))
        for i in ordered_indices:
            if i in f:
                continue
            #print(f)
            newF = np.concatenate([f, [i]]).astype(np.int64)
            newFeatures.append(newF)
            obj = computeObj(X, Y, newF, lam, mu)
            res.append(obj)
            if obj < 0:
                good_rules.append(newF)
                good_rule_obj.append(obj)
    print('hey')
    feature_set = np.array(newFeatures)[np.argsort(res)][:numKeep]
    #np.array(newFeatures)[[0,1,2,3]]
print(good_rule_obj)

[]
1
hey
[125]
20
[124]
20
[123]
20
[111]
20
[162]
20
[126]
20
[112]
20
[163]
20
[110]
20
[113]
20
[161]
20
[114]
20
[36]
20
[160]
20
[109]
20
[137]
20
[127]
20
[115]
20
[37]
20
[35]
20
hey
[109 123]
20
[123 109]
20
[123 110]
20
[110 123]
20
[123  71]
20
[125  71]
20
[123  91]
20
[125 107]
20
[125 108]
20
[125  91]
20
[123 108]
20
[124 108]
20
[125 133]
20
[125  95]
20
[125 129]
20
[125  43]
20
[125  50]
20
[125 134]
20
[124  71]
20
[125 124]
20
hey
[123 109  74]
20
[109 123  74]
20
[123 109  71]
20
[109 123  91]
20
[123 109  91]
20
[109 123  71]
20
[123  71 109]
20
[123  71  13]
20
[123  91 109]
20
[123  91  13]
20
[123 110 129]
20
[123  71 110]
20
[123 109 108]
20
[123  91 110]
20
[109 123  50]
20
[109 123  43]
20
[109 123 108]
20
[123 109 107]
20
[109 123 133]
20
[123 110  91]
20
hey
[109 123  74  91]
20
[123 109  71  74]
20
[123 109  74  71]
20
[109 123  74  71]
20
[123 109  74  91]
20
[123 109  91  74]
20
[109 123  91  74]
20
[123  91 109  74]
20
[109 123  71  74]
20
[123  71 109 

In [19]:
np.array(good_rules)[np.argsort(good_rule_obj)]

array([array([109, 123,  74,  91,  93]), array([123, 109,  91,  74,  95]),
       array([123, 109,  91,  74,  94]), ..., array([109,  63]),
       array([160,  28]), array([ 36, 149])], dtype=object)